In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1, "../python/functions")
sys.path.insert(2, "../python/architecture")

In [3]:
from data_prep_functions import mnist_prep
from model_functions import *
from plotting_functions import *
# from new_Adam import *
import no_gpu
import reproducible

In [4]:
import time

In [5]:
X_train, X_val, y_train, y_val = mnist_prep()

In [6]:
# use samller dataset for increased speed
X_train_small = X_train[:1000, :]
X_val_small = X_val[:500, :]
y_train_small = y_train[:1000]
y_val_small = y_val[:500]

n_cols = X_train_small.shape[1]

In [12]:
model = nn_model_structure(layers = 5,
                           neurons = [128, 128, 64, 32, 10],
                           n_cols = X_train_small.shape[1])
model = nn_model_compile(model,
                         optimizer = "adam")

In [ ]:
start_time = time.time()
model = nn_model_fit(model,
                     X_train = X_train_small,
                     y_train = y_train_small,
                     X_val = X_val_small,
                     y_val = y_val_small,
                     batch_size = 100,
                     epochs = 10)
end_time = time.time()
print("Overall calculation took {} seconds.".format(end_time - start_time))
model.summary()
nn_plot_acc(model)
# nn_save(model, "../models/model_test_small.h5")

In [13]:
true_acc = model.evaluate(X_val_small, y_val_small)[1]
print(true_acc)

shape_0 = model.get_weights()[0].shape
print(shape_0)

old_first_weights = model.get_weights()[0]

# first_weights = [np.random.normal(size = shape_0)]
first_weights = [model.get_weights()[0]-1]
print(first_weights)
print(first_weights[0].shape)

all_weights = first_weights + model.get_weights()[1:]
# print(all_weights)

model.set_weights(all_weights)

fake_acc = model.evaluate(X_val_small, y_val_small)[1]
print(fake_acc)

old_weights = [old_first_weights] + model.get_weights()[1:]

model.set_weights(old_weights)

old_true_acc = model.evaluate(X_val_small, y_val_small)[1]
print(old_true_acc)

16/16 [==============================] - 0s 2ms/step - loss: 2.3362 - accuracy: 0.0900
0.09000000357627869
(784, 128)
[array([[-0.97046286, -0.95202035, -1.012306  , ..., -0.9541457 ,
        -0.96590805, -1.0241356 ],
       [-0.98207283, -1.0065734 , -0.97689056, ..., -0.8966991 ,
        -0.94533825, -1.0408244 ],
       [-1.0305612 , -1.0306166 , -0.9585712 , ..., -1.0205923 ,
        -0.94706   , -0.99042463],
       ...,
       [-1.0070465 , -0.93766457, -1.0177236 , ..., -1.0898496 ,
        -0.92858386, -0.94916195],
       [-1.0271006 , -0.9543718 , -1.0279711 , ..., -1.081348  ,
        -0.9017614 , -0.9469036 ],
       [-0.9150654 , -1.0163006 , -1.0079986 , ..., -0.9525798 ,
        -1.0648855 , -0.941217  ]], dtype=float32)]
(784, 128)
16/16 [==============================] - 0s 2ms/step - loss: 2.3156 - accuracy: 0.0900
0.09000000357627869
16/16 [==============================] - 0s 3ms/step - loss: 2.3362 - accuracy: 0.0900
0.09000000357627869


# EnKF

In [72]:
init_model = nn_model_structure(layers = layers,
                                neurons = neurons,
                                n_cols = n_cols)
model = nn_model_compile(init_model,
                         optimizer = "sgd")

In [73]:
for params in model.get_weights():
    print(params.shape)

(784, 128)
(128,)
(128, 128)
(128,)
(128, 64)
(64,)
(64, 32)
(32,)
(32, 10)
(10,)


In [74]:
weights = model.get_weights()

In [75]:
np.append(np.array(weights[1].shape),0)

array([128,   0])

In [76]:
shapes = [np.array(params.shape) for params in weights]
shape_elements = [np.prod(shape) for shape in shapes]

In [77]:
shapes

[array([784, 128]),
 array([128]),
 array([128, 128]),
 array([128]),
 array([128,  64]),
 array([64]),
 array([64, 32]),
 array([32]),
 array([32, 10]),
 array([10])]

In [78]:
shape_elements

[100352, 128, 16384, 128, 8192, 64, 2048, 32, 320, 10]

In [79]:
X_train_b1 = X_train[0:250] 
y_train_b1 = y_train[0:250] 
X_train_b2 = X_train[250:500] 
y_train_b2 = y_train[250:500]
X_train_b3 = X_train[500:750] 
y_train_b3 = y_train[500:750] 
X_train_b4 = X_train[750:1000] 
y_train_b4 = y_train[750:1000]
# batch_size * 0, ..., n/batch_size

In [80]:
len(X_train_b1)

250

In [81]:
y_pred = model.predict(X_train_b1)

In [121]:
y_pred

array([[0.0619125 , 0.06424606, 0.04867891, ..., 0.0943555 , 0.09618338,
        0.12829606],
       [0.05875716, 0.06647606, 0.0357979 , ..., 0.09903294, 0.10037428,
        0.10961606],
       [0.06155263, 0.06351205, 0.04611754, ..., 0.10116713, 0.08640555,
        0.11951485],
       ...,
       [0.06788904, 0.06239324, 0.0487645 , ..., 0.0876164 , 0.09868892,
        0.14136861],
       [0.06463828, 0.06525789, 0.05138935, ..., 0.10799036, 0.0952318 ,
        0.14591932],
       [0.06696405, 0.06082284, 0.04003751, ..., 0.0973183 , 0.12084081,
        0.11761768]], dtype=float32)

In [83]:
y_pred.shape

(250, 10)

In [86]:
np.array(y_train_b1).shape

(250, 10)

In [89]:
np.multiply(np.array([[1, 2],[3, 4]]),np.array([[5, 6],[7, 8]]))

array([[ 5, 12],
       [21, 32]])

In [90]:
np.log([[1, np.e],[np.e**2, 0]])

array([[  0.,   1.],
       [  2., -inf]])

In [99]:
np.sum(np.multiply(np.array(y_train_b1), np.log(y_pred))) # Frobenius-Skalarprodukt

-601.422

In [114]:
import tensorflow as tf

loss_fct = tf.keras.losses.CategoricalCrossentropy(reduction = "none")

y_train_b1_tf = tf.constant(y_train_b1)
y_pred_tf = tf.Variable(y_pred)
with tf.GradientTape() as tape:
    loss_value = loss_fct(y_train_b1_tf, y_pred_tf)
    
jacobian_tf = tape.gradient(loss_value, y_pred_tf)

In [119]:
jacobian_tf.numpy()[0]

array([ 1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
       -5.0690026,  1.       ,  1.       ,  1.       ,  1.       ],
      dtype=float32)

#### Das stimmt aber nicht wirklich. In meiner Arbeit habe ich da analog zum Paper andere Werte!!!

In [128]:
np.array(y_train_b1)

(250, 10)

In [130]:
(1/y_pred)[0]

array([16.151829 , 15.565157 , 20.542776 ,  6.013862 , 17.11169  ,
        6.0690026,  8.559167 , 10.598216 , 10.396807 ,  7.7944713],
      dtype=float32)

In [131]:
for i in y_pred[0]:
    print(1/i)

16.151828312136058
15.565157208197649
20.54277621142367
6.013862317232082
17.11169073238116
6.069002589430677
8.559166511066012
10.598216181528043
10.396806386607087
7.794471487907447


In [143]:
jacobian = (-1) * np.multiply(np.array(y_train_b1), np.array(1 / (y_pred + delta)))

In [144]:
jacobian[0]

array([-0.       , -0.       , -0.       , -0.       , -0.       ,
       -5.8902626, -0.       , -0.       , -0.       , -0.       ],
      dtype=float32)

In [164]:
jacobian.shape

(250, 10)

#### Das sieht schon eher nach der Jacobi-Matrix aus.

## Für mehrere Partikel

In [235]:
np.mean(list(y_pred_dict.values()), axis = 0) # shape is again (250, 10)

array([[0.13080385, 0.12461238, 0.0669295 , ..., 0.08343228, 0.10461207,
        0.07551399],
       [0.1205229 , 0.13815689, 0.06966151, ..., 0.08500493, 0.09613462,
        0.07695267],
       [0.12284128, 0.11670828, 0.07473157, ..., 0.08850323, 0.10310457,
        0.08063252],
       ...,
       [0.12565033, 0.12483213, 0.07505317, ..., 0.08072939, 0.10383433,
        0.07880531],
       [0.1286582 , 0.13134146, 0.07118312, ..., 0.08829232, 0.09477226,
        0.07336508],
       [0.11979757, 0.15881512, 0.06772456, ..., 0.08269005, 0.09318483,
        0.07528629]], dtype=float32)

In [236]:
y_pred_mean = np.mean(list(y_pred_dict.values()), axis = 0)

In [237]:
k=j=1

In [238]:
y_pred_centered = y_pred_dict["model_{}".format(str(k+1))] - y_pred_mean

In [239]:
y_pred_centered.shape

(250, 10)

In [240]:
jacobian_dict["model_{}".format(str(j+1))].shape

(250, 10)

In [241]:
np.sum(np.multiply(y_pred_centered, jacobian_dict["model_{}".format(str(j+1))]))

14.36283

In [242]:
d = np.zeros(shape = (particles, particles))
for k in range(particles):
    y_pred_centered = y_pred_dict["model_{}".format(str(k+1))] - y_pred_mean
    for j in range(particles):
        d[k][j] = np.sum(np.multiply(y_pred_centered, jacobian_dict["model_{}".format(str(j+1))]))

In [243]:
d.shape

(5, 5)

In [244]:
d

array([[ 11.72108269,  -3.31927323, -10.74712086, -11.9116497 ,
        -13.93153191],
       [ -0.65922785,  14.36283016, -14.23204803,  -1.64433455,
         -3.21250176],
       [-11.05075359, -11.67054176,  40.39073181, -15.71726036,
        -18.98975754],
       [  1.43578172,   4.34402227,   1.23750496,  29.08330917,
         10.86227608],
       [ -1.44689035,  -3.71704483, -16.64907074,   0.1899274 ,
         25.27150917]])

In [245]:
h_t = h_0 / (np.sqrt(np.sum(d**2)) + epsilon)

In [246]:
h_t

0.013560187587328335

In [247]:
weights_dict["model_1"]

[array([[ 0.07713559,  0.0517761 , -0.00964706, ...,  0.10081124,
          0.04777463,  0.00378315],
        [-0.04173185,  0.01739302, -0.10093844, ...,  0.01061524,
         -0.0033663 ,  0.09081071],
        [-0.0628881 , -0.02058959, -0.0443719 , ..., -0.02271051,
          0.01960537, -0.06349264],
        ...,
        [-0.02446294,  0.01684814,  0.05190816, ..., -0.06279122,
         -0.09258976, -0.07137857],
        [-0.00366583, -0.04198691, -0.04249102, ..., -0.06405497,
         -0.01433972,  0.01016243],
        [ 0.07003888, -0.03218133, -0.01340503, ...,  0.00068751,
         -0.00414753,  0.04405415]], dtype=float32),
 array([ 0.03794942,  0.17547506, -0.02546689,  0.12550451, -0.17037669,
         0.04806631,  0.0251469 ,  0.14447346, -0.0517147 , -0.09859668,
         0.00565057, -0.08521851,  0.04422217,  0.05085195, -0.00641812,
         0.12281357,  0.017707  ,  0.06824613,  0.06653562,  0.03970008,
        -0.09117553, -0.01378166, -0.02946344,  0.16246028,  0.034

In [294]:
for i, w_b in enumerate(weights_dict["model_1"]):
    weights_dict["model_1"][i] = np.reshape(w_b, (1, -1))
    print(i)
    print(w_b)

0
[[-0.0574204   0.03053112 -0.07511065 ... -0.0001479  -0.02547782
  -0.03556966]]
1
[ 0.17799328  0.08427169  0.05014688 -0.07709871 -0.06837214 -0.03103684
  0.13334014 -0.00727592  0.03098815 -0.04806816  0.14077161  0.16480273
 -0.02731767 -0.12642887  0.03413132  0.01107246  0.15725535  0.03570703
 -0.05508201  0.09659226 -0.02733269 -0.02736649  0.05519364 -0.00527646
  0.04162928 -0.00176206  0.02653918  0.04177421  0.07928815 -0.0331973
  0.031421    0.09675165 -0.01660075  0.18980758  0.05560138 -0.12539357
  0.00268483  0.0444257   0.02294068  0.00963581 -0.01576732  0.00199714
 -0.07055862  0.00722155  0.03520704 -0.1161338   0.02940669  0.05549415
 -0.04540783 -0.05826958  0.15189765 -0.03939407  0.02894199  0.03825189
  0.04077317  0.09832264  0.09093892  0.10130176 -0.01401393  0.02790483
  0.07081012  0.08541103  0.08526853 -0.13657218 -0.05489184  0.03574394
  0.014156    0.00669313 -0.06562077 -0.0157997  -0.0019041  -0.01974379
  0.10342217  0.05798227  0.10752979  0

In [249]:
for w_b in weights_dict["model_1"]:
    print(w_b.shape)

(1, 100352)
(1, 128)
(1, 16384)
(1, 128)
(1, 8192)
(1, 64)
(1, 2048)
(1, 32)
(1, 320)
(1, 10)


In [250]:
type(weights_dict["model_1"])

list

In [265]:
weights_vector = np.concatenate(weights_dict["model_1"], axis = 1)

In [266]:
weights_vector

array([[ 0.07713559,  0.0517761 , -0.00964706, ...,  0.1345713 ,
         0.32843778,  0.21633522]], dtype=float32)

In [267]:
weights_vector.shape

(1, 127658)

In [271]:
shape_elements

array([     0, 100352, 100480, 116864, 116992, 125184, 125248, 127296,
       127328, 127648, 127658], dtype=int32)

In [253]:
shapes

[array([784, 128]),
 array([128]),
 array([128, 128]),
 array([128]),
 array([128,  64]),
 array([64]),
 array([64, 32]),
 array([32]),
 array([32, 10]),
 array([10])]

In [257]:
tuple(shapes[0])

(784, 128)

In [287]:
for i in range(len(shape_elements)-1):
    print(np.reshape(weights_vector[0][shape_elements[i]:shape_elements[i+1]], tuple(shapes[i])).shape)

(784, 128)
(128,)
(128, 128)
(128,)
(128, 64)
(64,)
(64, 32)
(32,)
(32, 10)
(10,)


In [275]:
i=0
weights_vector[(shape_elements[i]):(shape_elements[i+1])].shape

(1, 127658)

In [276]:
shape_elements[1]

100352

In [281]:
i=0
start = shape_elements[i]
end = shape_elements[i+1]

weights_vector[start:end].shape

(1, 127658)

In [286]:
weights_vector[0]

array([ 0.07713559,  0.0517761 , -0.00964706, ...,  0.1345713 ,
        0.32843778,  0.21633522], dtype=float32)

In [388]:
for f in range(len(weights_dict["model_1"])):
    print(weights_dict["model_1"][f].shape)

(784, 128)
(128,)
(128, 128)
(128,)
(128, 64)
(64,)
(64, 32)
(32,)
(32, 10)
(10,)


In [379]:
len(shape_elements)-1

10

In [380]:
l=0
start = shape_elements[l]
end = shape_elements[l+1]

In [381]:
start, end

(0, 100352)

In [383]:
i=0
weights_vector_dict["model_{}".format(str(i+1))][start:end].shape

(100352,)

In [364]:
len(weights_dict["model_{}".format(str(i+1))])

10

In [347]:
np.array(list(weights_vector_dict.values())[0]).ravel()

array([-0.03483577,  0.08977479,  0.0743715 , ...,  0.41685635,
        0.06919966, -0.3420352 ], dtype=float32)

In [354]:
np.array(list(weights_vector_dict.values())).shape

(5, 127658)

In [341]:
np.reshape(np.array([2,2]), (1, 2))

array([[2, 2]])

In [315]:
weights_all_ptcls = np.array(list(weights_vector_dict.values()))

In [358]:
weights_all_ptcls

array([[-0.0822654 ,  0.04185335,  0.00127433, ..., -0.07733928,
         0.41856736,  0.23792657],
       [-0.07878807,  0.02858039,  0.03373574, ...,  0.5289479 ,
        -0.57863945, -0.16768108],
       [-0.0177621 , -0.00091849, -0.10263124, ...,  0.23254985,
        -0.35255197, -0.22358526],
       [-0.04534946,  0.05136504,  0.0665621 , ..., -0.6015446 ,
         0.18586604,  0.57174736],
       [-0.01346172, -0.02157436,  0.01849296, ..., -0.03406319,
        -0.2861168 , -0.51725864]], dtype=float32)

In [323]:
np.reshape(weights_all_ptcls, (5, 127658)).shape

(5, 127658)

In [300]:
type(weights_vector_dict["model_1"])

numpy.ndarray

In [304]:
np.array([np.array([1,2]),np.array([3,4])])

array([[1, 2],
       [3, 4]])

In [312]:
d[0:2,0:2]

array([[ 21.08625031, -17.41467094],
       [ -4.93830538,  22.85575485]])

In [314]:
np.matmul(d[:2,:2], np.array([np.array([1,2]),np.array([3,4])])) # richtige Matrixmultiplikation

array([[-31.15776253, -27.48618317],
       [ 63.62895918,  81.54640865]])

In [361]:
weights_all_ptcls.shape

(5, 127658)

### Batches bestimmen

In [417]:
batch_size = 99

In [419]:
n = len(X_train)
num_batches = int(np.ceil(n / batch_size))
batch_indices = np.cumsum([0] + list(np.ones(num_batches) * batch_size))
batch_indices[-1] = n

In [420]:
batch_indices

array([   0.,   99.,  198.,  297.,  396.,  495.,  594.,  693.,  792.,
        891.,  990., 1000.])

In [424]:
X_batches = [X_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]
y_batches = [y_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]

In [425]:
for batch in y_batches:
    print(batch.shape)

(99, 10)
(99, 10)
(99, 10)
(99, 10)
(99, 10)
(99, 10)
(99, 10)
(99, 10)
(99, 10)
(99, 10)
(10, 10)


In [426]:
len(X_batches)

11

# Ab hier wird es spannend

In [436]:
X_train = X_train_small
X_test = X_val_small
y_train = y_train_small
y_test = y_val_small

In [437]:
batch_size = 250
epochs = 3
particles = 5
early_stopping = 0.001
batch_normal = False

In [438]:
layers = 5
neurons = [128, 128, 64, 32, 10]
n_cols = X_train.shape[1]

In [449]:
delta = 0.005
h_0 = 300
epsilon = 0.005

In [440]:
n = len(X_train)
num_batches = int(np.ceil(n / batch_size))
batch_indices = np.cumsum([0] + list(np.ones(num_batches) * batch_size))
batch_indices[-1] = n

In [441]:
X_batches = [X_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]
y_batches = [y_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]

In [442]:
# start_time = time.time()

In [443]:
model_dict = {}
weights_dict = {}
y_pred_dict = {}
jacobian_dict = {}
weights_vector_dict = {}

init_model = nn_model_structure(layers = layers,
                                neurons = neurons,
                                n_cols = n_cols)
init_model = nn_model_compile(init_model,
                              optimizer = "sgd")
weights = init_model.get_weights()
shapes = [np.array(params.shape) for params in weights]
shape_elements = np.cumsum([0] + [np.prod(shape) for shape in shapes])

In [450]:
for i in range(particles):
    # just an initial model with the correct structure regarding neurons, layers, activation functions, Glorot initialization
    model = nn_model_structure(layers = layers,
                               neurons = neurons,
                               n_cols = n_cols)
    model = nn_model_compile(model,
                             optimizer = "sgd")
    # for every particle write the model in a dictionary
    model_dict["model_{}".format(str(i+1))] = model
    
    # for every particles write the weights and biases in a dictionary
    weights_dict["model_{}".format(str(i+1))] = model_dict["model_{}".format(str(i+1))]\
                                                    .get_weights()

In [445]:
for i in range(particles):
    print(model_dict["model_{}".format(str(i+1))].evaluate(X_val_small, y_val_small)[1])

16/16 [==============================] - 0s 3ms/step - loss: 2.4076 - accuracy: 0.0860
[2.4076390266418457, 0.0860000029206276]
16/16 [==============================] - 0s 2ms/step - loss: 2.3925 - accuracy: 0.0940
[2.392542839050293, 0.09399999678134918]
16/16 [==============================] - 0s 3ms/step - loss: 2.4388 - accuracy: 0.1120
[2.438811779022217, 0.1120000034570694]
16/16 [==============================] - 0s 3ms/step - loss: 2.4079 - accuracy: 0.0640
[2.4079365730285645, 0.06400000303983688]
16/16 [==============================] - 0s 3ms/step - loss: 2.4230 - accuracy: 0.0800
[2.423008441925049, 0.07999999821186066]


In [451]:
# loop over all epochs
for epoch in range(epochs):
    # loop over all batches
    for b in range(len(X_batches)):
        for i in range(particles):
            # for every particle write the predictions on the training batches in a dictionary
            y_pred_dict["model_{}".format(str(i+1))] = model_dict["model_{}".format(str(i+1))]\
                                                            .predict(X_batches[b])
            
            # for every particle write the Jacobian in a dictionary
            jacobian_dict["model_{}".format(str(i+1))] = (-1) * np.multiply(np.array(y_batches[b]), 
                                                                            np.array(1 / (y_pred_dict["model_{}".format(str(i+1))] + delta)))
        
        # compute the mean of the predictions
        y_pred_mean = np.mean(list(y_pred_dict.values()), axis = 0)

        # compute the matrix D elementwise
        d = np.zeros(shape = (particles, particles))
        for k in range(particles):
            y_pred_centered = y_pred_dict["model_{}".format(str(k+1))] - y_pred_mean
            for j in range(particles):
                d[k][j] = np.sum(np.multiply(y_pred_centered, jacobian_dict["model_{}".format(str(j+1))]))

        # compute the scalar h_t
        h_t = h_0 / (np.sqrt(np.sum(d**2)) + epsilon)

        # Reshape the weights and biases so that they are no longer matrices and vectores, but now one single vector
        for i in range(particles):
            for j, w_b in enumerate(weights_dict["model_{}".format(str(i+1))]):
                weights_dict["model_{}".format(str(i+1))][j] = np.reshape(w_b, (1, -1))
            weights_vector_dict["model_{}".format(str(i+1))] = np.concatenate(weights_dict["model_{}".format(str(i+1))], axis = 1).ravel()
        
        # matrix with particle parameters as row vectors
        weights_all_ptcls = np.array(list(weights_vector_dict.values()))

        # compute the matrix with the updates for each particle
        weights_all_ptcls = weights_all_ptcls - h_t * np.matmul(d, weights_all_ptcls)

        for i in range(particles):
            # write the updates back into the dictionary
            weights_vector_dict["model_{}".format(str(i+1))] = weights_all_ptcls[i]
            # reshape the updates, so that they are of the original matrx and vector shape
            for l in range(len(shape_elements)-1):
                start = shape_elements[l]
                end = shape_elements[l+1]
                weights_dict["model_{}".format(str(i+1))][l] = np.reshape(weights_vector_dict["model_{}".format(str(i+1))][start:end], tuple(shapes[l]))
            # set new weights for model
            model_dict["model_{}".format(str(i+1))].set_weights(weights_dict["model_{}".format(str(i+1))])

In [452]:
for i in range(particles):
    print(model_dict["model_{}".format(str(i+1))].evaluate(X_val_small, y_val_small)[1])

16/16 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0840
[nan, 0.08399999886751175]
16/16 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0840
[nan, 0.08399999886751175]
16/16 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0840
[nan, 0.08399999886751175]
16/16 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0840
[nan, 0.08399999886751175]
16/16 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0840
[nan, 0.08399999886751175]


In [448]:
# end_time = time.time()
# print("Overall calculation took {} seconds.".format(end_time - start_time))